In [13]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split


In [8]:
data_main = pd.read_csv(
    './datasets/tradingview/bitcoin/BTCUSD_1D_to_2023-03-11.csv',
    delimiter=',',
    low_memory=False
)

In [9]:
data_main

,time,open,high,low,close,Histogram,MACD,Signal,RSI,RSI-based MA,Upper Bollinger Band,Lower Bollinger Band,OnBalanceVolume,Smoothing Line,ADX,NV
0,2009-10-05T04:00:00+04:00,0.000764,0.000764,0.000764,0.000764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-10-06T04:00:00+04:00,0.000885,0.000885,0.000885,0.000885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,0.000000
2,2009-10-07T04:00:00+04:00,0.001050,0.001050,0.001050,0.001050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,0.000000
3,2009-10-08T04:00:00+04:00,0.001084,0.001084,0.001084,0.001084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,0.000000
4,2009-10-09T04:00:00+04:00,0.001200,0.001200,0.001200,0.001200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4803,2023-03-07T03:00:00+03:00,22415.570000,22552.120000,21927.510000,22201.830000,-238.134300,-109.924026,128.210274,41.678396,49.566643,NaN,NaN,7.408373e+06,7.426450e+06,22.036393,-15118.130417
4804,2023-03-08T03:00:00+03:00,22203.580000,22282.920000,21588.560000,21701.470000,-258.385641,-194.771777,63.613864,37.572144,48.014107,NaN,NaN,7.390680e+06,7.417858e+06,21.677303,-17693.066945
4805,2023-03-09T03:00:00+03:00,21699.330000,21842.130000,19858.830000,20346.900000,-344.559235,-367.085180,-22.525945,29.188244,46.027901,NaN,NaN,7.358888e+06,7.403712e+06,23.090552,-31792.011157
4806,2023-03-10T03:00:00+03:00,20349.670000,20350.450000,19559.360000,20222.680000,-388.230876,-507.814539,-119.583664,28.558888,44.464712,NaN,NaN,7.313238e+06,7.378934e+06,24.620602,-45649.645680


In [10]:
#DATE = 'Date'
TIME = 'time'
OPEN = 'open'
HIGH = 'high'
LOW = 'low'
CLOSE = 'close'
VOLUME = 'volume'
#MARKET_CAP = 'Market cap'

#INDICATORS
MOVING_AVERAGE = 'MA'
SMOOTHING_LINE = 'Smoothing Line'
HISTOGRAM = 'Histogram'
MACD = 'MACD'
SIGNAL = 'Signal'
RSI = 'RSI'
RSI_MA = 'RSI-based MA'
UP_BOL_BAND = 'Upper Bollinger Band'
LOW_BOL_BAND = 'Lower Bollinger Band'
OBV = 'OnBalanceVolume'
ADX = 'ADX'
NV = 'NV'

ALL_COLUMNS = {
    TIME, OPEN, HIGH, LOW, CLOSE,
    MOVING_AVERAGE, SMOOTHING_LINE, HISTOGRAM, MACD, SIGNAL, RSI, RSI_MA, UP_BOL_BAND, LOW_BOL_BAND, OBV, ADX
}


In [25]:


class DatasetPreparator:
    def __init__(self, dataset):
        self.dataset: DataFrame = dataset.copy(deep=True)


    def prepare(self):
        self.remove_columns()
        self.transform_types()
        self.remove_nan()
        return self.dataset


    def preprocess(self):
        self.optimize_values([OPEN, HIGH, LOW, CLOSE, HISTOGRAM, MACD, SIGNAL, RSI, RSI_MA, OBV, SMOOTHING_LINE, ADX, NV])
        return self.dataset

    def get_data_for_prediction(self, test_size=0.1):
        self.prepare()
        self.preprocess()
        self.remove_columns_for_prediction()
        x_train, x_test, y_train, y_test = train_test_split(
            self.dataset,
            self.dataset[CLOSE],
            test_size=test_size,
            shuffle=False
        )
        return x_train.to_numpy(), x_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()



    def transform_types(self):
        try:
            self.dataset[TIME] = pd.to_datetime(self.dataset[TIME],unit='s')
        except ValueError:
            self.dataset[TIME] = pd.to_datetime(self.dataset[TIME], utc=True)

    def remove_columns(self):
        unused_columns = [
            UP_BOL_BAND, LOW_BOL_BAND
        ]
        self.dataset= self.dataset.drop(unused_columns, axis=1)

    def remove_columns_for_prediction(self):
        self.dataset= self.dataset.drop([TIME], axis=1)

    @staticmethod
    def plot(dataset):
        # data_plot = self.dataset[[TIME, CLOSE]]
        # data_plot.plot()
        ax = dataset.plot(x=TIME, y=CLOSE)
        ax.set_xlabel("Date")
        ax.set_ylabel("Close Price (USD)")

    def remove_nan(self):
        self.dataset = self.dataset.fillna(0.0)


    def optimize_values(self, columns):
        scaler = MinMaxScaler(feature_range=(0,1))
        self.dataset[columns] = scaler.fit_transform(self.dataset[columns])

    def reverse_data(self, arr):
        scaler = MinMaxScaler(feature_range=(0,1))
        close_price = self.dataset[CLOSE].values.reshape(-1, 1)
        print(close_price)
        scaler.fit_transform(close_price)
        return scaler.inverse_transform(arr)


preparator = DatasetPreparator(dataset=data_main)
#dataset = preparator.prepare()
#dataset
#DatasetPreparator.plot(dataset)
x_train, x_test, y_train, y_test = preparator.get_data_for_prediction()

In [14]:
np.save( './datasets/prepared/btc/1d_with_indicators/v1/x_train.npy', x_train)
np.save( './datasets/prepared/btc/1d_with_indicators/v1/x_test.npy', x_test)
np.save( './datasets/prepared/btc/1d_with_indicators/v1/y_train.npy', y_train)
np.save( './datasets/prepared/btc/1d_with_indicators/v1/y_test.npy', y_test)

In [46]:
dataset

,time,open,high,low,close,Histogram,MACD,Signal,RSI,RSI-based MA,OnBalanceVolume,Smoothing Line,ADX,NV
0,2009-10-05 00:00:00+00:00,0.000764,0.000764,0.000764,0.000764,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
1,2009-10-06 00:00:00+00:00,0.000885,0.000885,0.000885,0.000885,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
2,2009-10-07 00:00:00+00:00,0.001050,0.001050,0.001050,0.001050,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
3,2009-10-08 00:00:00+00:00,0.001084,0.001084,0.001084,0.001084,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
4,2009-10-09 00:00:00+00:00,0.001200,0.001200,0.001200,0.001200,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4803,2023-03-07 00:00:00+00:00,22415.570000,22552.120000,21927.510000,22201.830000,-238.134300,-109.924026,128.210274,41.678396,49.566643,7.408373e+06,7.426450e+06,22.036393,-15118.130417
4804,2023-03-08 00:00:00+00:00,22203.580000,22282.920000,21588.560000,21701.470000,-258.385641,-194.771777,63.613864,37.572144,48.014107,7.390680e+06,7.417858e+06,21.677303,-17693.066945
4805,2023-03-09 00:00:00+00:00,21699.330000,21842.130000,19858.830000,20346.900000,-344.559235,-367.085180,-22.525945,29.188244,46.027901,7.358888e+06,7.403712e+06,23.090552,-31792.011157
4806,2023-03-10 00:00:00+00:00,20349.670000,20350.450000,19559.360000,20222.680000,-388.230876,-507.814539,-119.583664,28.558888,44.464712,7.313238e+06,7.378934e+06,24.620602,-45649.645680


In [52]:
tscv = TimeSeriesSplit(n_splits=5)

for i, (train_index, test_index) in enumerate(tscv.split(dataset, dataset[CLOSE])):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

Fold 0:
  Train: index=[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240

In [22]:
predicted = np.array([[0.3319916 ],
                        [0.33135527],
                        [0.33069226],
                        [0.33004248],
                        [0.32951245],
                        [0.32935852],
                        [0.32892883],
                        [0.32905734],
                        [0.3295062 ],
                        [0.32988375],
                        [0.33020273],
                        [0.32962385],
                        [0.32828444],
                        [0.32669997],
                        [0.32541642],
                        [0.3242246 ],
                        [0.3240854 ],
                        [0.32403946],
                        [0.3241595 ],
                        [0.32356912],
                        [0.3227742 ],
                        [0.3221017 ],
                        [0.32184812],
                        [0.32183495],
                        [0.32199755],
                        [0.3220189 ],
                        [0.32211962],
                        [0.32236248],
                        [0.3223473 ],
                        [0.32225344],
                        [0.3223403 ],
                        [0.32266736],
                        [0.32284823],
                        [0.32273272],
                        [0.3228705 ],
                        [0.3228769 ],
                        [0.322695  ],
                        [0.32262418],
                        [0.3221644 ],
                        [0.32192165],
                        [0.3216119 ],
                        [0.32131314],
                        [0.32150355],
                        [0.32202712],
                        [0.32198605],
                        [0.32199943],
                        [0.32175913],
                        [0.32171458],
                        [0.32205102],
                        [0.32209286],
                        [0.32215282],
                        [0.32267338],
                        [0.3230416 ],
                        [0.32345402],
                        [0.3240214 ],
                        [0.32427546],
                        [0.32420897],
                        [0.32438657],
                        [0.32472917],
                        [0.32488275],
                        [0.32465395],
                        [0.32426223],
                        [0.3235131 ],
                        [0.32314354],
                        [0.32247654],
                        [0.3219262 ],
                        [0.32209414],
                        [0.3222754 ],
                        [0.32185164],
                        [0.321583  ],
                        [0.32150832],
                        [0.32121143],
                        [0.3214306 ],
                        [0.32138965],
                        [0.3211539 ],
                        [0.32097736],
                        [0.3206338 ],
                        [0.31982404],
                        [0.318913  ],
                        [0.31753436],
                        [0.31626007],
                        [0.3149645 ],
                        [0.31405887],
                        [0.3135739 ],
                        [0.31311288],
                        [0.3127922 ],
                        [0.31198972],
                        [0.3113175 ],
                        [0.3104554 ],
                        [0.30936766],
                        [0.30832845],
                        [0.30695397],
                        [0.30557975],
                        [0.30434576],
                        [0.30329084],
                        [0.30193612],
                        [0.30077603],
                        [0.29976523],
                        [0.29929385],
                        [0.29902837],
                        [0.29840204],
                        [0.29757956],
                        [0.29649964],
                        [0.295324  ],
                        [0.29385224],
                        [0.29244024],
                        [0.29175633],
                        [0.29118478],
                        [0.2908153 ],
                        [0.29028866],
                        [0.28949338],
                        [0.28797   ],
                        [0.28522265],
                        [0.28226247],
                        [0.27974036],
                        [0.27767822],
                        [0.2755245 ],
                        [0.27352402],
                        [0.27215323],
                        [0.2716052 ],
                        [0.27124628],
                        [0.2710519 ],
                        [0.27116632],
                        [0.27152562],
                        [0.2720611 ],
                        [0.2724248 ],
                        [0.2723716 ],
                        [0.27202418],
                        [0.27116877],
                        [0.27030283],
                        [0.26922104],
                        [0.26784468],
                        [0.26621306],
                        [0.26501787],
                        [0.26390633],
                        [0.2627174 ],
                        [0.26185814],
                        [0.26092297],
                        [0.25991747],
                        [0.25864393],
                        [0.2569585 ],
                        [0.25497633],
                        [0.25315487],
                        [0.25160864],
                        [0.25026223],
                        [0.249175  ],
                        [0.24815099],
                        [0.247412  ],
                        [0.24686274],
                        [0.2462513 ],
                        [0.2457002 ],
                        [0.24489611],
                        [0.24395643],
                        [0.24311706],
                        [0.24169391],
                        [0.24014276],
                        [0.23916191],
                        [0.23882319],
                        [0.23866944],
                        [0.23852089],
                        [0.23810156],
                        [0.23754232],
                        [0.23694007],
                        [0.2361489 ],
                        [0.23533158],
                        [0.23466738],
                        [0.23392142],
                        [0.23303674],
                        [0.23246719],
                        [0.23154   ],
                        [0.2308949 ],
                        [0.23040795],
                        [0.22991885],
                        [0.22968553],
                        [0.22970186],
                        [0.22957626],
                        [0.22981761],
                        [0.22943637],
                        [0.22888555],
                        [0.22840874],
                        [0.22758006],
                        [0.22699018],
                        [0.22672352],
                        [0.22687905],
                        [0.22714281],
                        [0.22769548],
                        [0.22789885],
                        [0.22808632],
                        [0.22807662],
                        [0.2282548 ],
                        [0.22863717],
                        [0.22898296],
                        [0.22927384],
                        [0.22953333],
                        [0.22958006],
                        [0.22940105],
                        [0.22921169],
                        [0.22873163],
                        [0.22807398],
                        [0.22701797],
                        [0.22629161],
                        [0.22558995],
                        [0.22461145],
                        [0.22382835],
                        [0.2220925 ],
                        [0.22004437],
                        [0.2178046 ],
                        [0.21563227],
                        [0.21393622],
                        [0.2124946 ],
                        [0.21101032],
                        [0.20958044],
                        [0.20856722],
                        [0.20747721],
                        [0.2069963 ],
                        [0.20611022],
                        [0.2053837 ],
                        [0.20470677],
                        [0.20453915],
                        [0.2041021 ],
                        [0.20384927],
                        [0.20376055],
                        [0.2036442 ],
                        [0.20340405],
                        [0.2032257 ],
                        [0.20331942],
                        [0.20355095],
                        [0.20375179],
                        [0.20401953],
                        [0.20425195],
                        [0.2045175 ],
                        [0.20451073],
                        [0.20440975],
                        [0.20427893],
                        [0.20421143],
                        [0.20425706],
                        [0.20449923],
                        [0.20498276],
                        [0.20573515],
                        [0.20656253],
                        [0.20734286],
                        [0.20800647],
                        [0.20859413],
                        [0.20894755],
                        [0.20933455],
                        [0.20940042],
                        [0.20942932],
                        [0.20961611],
                        [0.20974521],
                        [0.20999657],
                        [0.21033013],
                        [0.21075739],
                        [0.21108545],
                        [0.21119696],
                        [0.21114537],
                        [0.2107493 ],
                        [0.21040903],
                        [0.21049888],
                        [0.21071337],
                        [0.2108803 ],
                        [0.21039508],
                        [0.20910041],
                        [0.20779623],
                        [0.20689058],
                        [0.20615181],
                        [0.20559357],
                        [0.20525374],
                        [0.20487022],
                        [0.20459262],
                        [0.204216  ],
                        [0.20384102],
                        [0.20339192],
                        [0.2025508 ],
                        [0.20152631],
                        [0.20051552],
                        [0.19976039],
                        [0.19920464],
                        [0.19886112],
                        [0.19844536],
                        [0.19781329],
                        [0.19688407],
                        [0.19596669],
                        [0.19515269],
                        [0.19463295],
                        [0.19419895],
                        [0.19371822],
                        [0.19331859],
                        [0.19292937],
                        [0.19238454],
                        [0.19204627],
                        [0.19173822],
                        [0.19142106],
                        [0.19125631],
                        [0.19112824],
                        [0.19089618],
                        [0.19100553],
                        [0.19122398],
                        [0.19130944],
                        [0.19070707],
                        [0.18992959],
                        [0.1891847 ],
                        [0.18843453],
                        [0.18801944],
                        [0.18778694],
                        [0.18753526],
                        [0.18747918],
                        [0.18739283],
                        [0.18738487],
                        [0.18761615],
                        [0.18773644],
                        [0.18776818],
                        [0.18784131],
                        [0.18775643],
                        [0.18777663],
                        [0.1878014 ],
                        [0.18794948],
                        [0.18778872],
                        [0.18796122],
                        [0.18796769],
                        [0.18778475],
                        [0.18759416],
                        [0.18745224],
                        [0.18704657],
                        [0.18633032],
                        [0.18547107],
                        [0.18496162],
                        [0.18510829],
                        [0.18551357],
                        [0.18605208],
                        [0.18627714],
                        [0.18659304],
                        [0.18685867],
                        [0.18701549],
                        [0.1873526 ],
                        [0.1878188 ],
                        [0.18831892],
                        [0.18879329],
                        [0.18921462],
                        [0.18965498],
                        [0.19022176],
                        [0.19091284],
                        [0.19156563],
                        [0.19265795],
                        [0.19350776],
                        [0.19426113],
                        [0.1950959 ],
                        [0.19573809],
                        [0.19643785],
                        [0.19708054],
                        [0.19748408],
                        [0.19761719],
                        [0.19783431],
                        [0.19833575],
                        [0.19860224],
                        [0.19840954],
                        [0.19818461],
                        [0.19814047],
                        [0.19823065],
                        [0.19837153],
                        [0.19944713],
                        [0.20087034],
                        [0.20172682],
                        [0.20278059],
                        [0.20357431],
                        [0.20457035],
                        [0.20525779],
                        [0.20548911],
                        [0.20597534],
                        [0.20625086],
                        [0.20638491],
                        [0.2066441 ],
                        [0.20702311],
                        [0.20754386],
                        [0.20808043],
                        [0.20867643],
                        [0.2091302 ],
                        [0.20960423],
                        [0.21015263],
                        [0.21076009],
                        [0.21157236],
                        [0.21222644],
                        [0.21217202]])

In [27]:
preparator.reverse_data(predicted)

[[2.22850629e-09]
 [4.01397575e-09]
 [6.46910729e-09]
 ...
 [3.01183615e-01]
 [2.99344856e-01]
 [3.02845336e-01]]


array([[0.3319916 ],
       [0.33135527],
       [0.33069226],
       [0.33004248],
       [0.32951245],
       [0.32935852],
       [0.32892883],
       [0.32905734],
       [0.3295062 ],
       [0.32988375],
       [0.33020273],
       [0.32962385],
       [0.32828444],
       [0.32669997],
       [0.32541642],
       [0.3242246 ],
       [0.3240854 ],
       [0.32403946],
       [0.3241595 ],
       [0.32356912],
       [0.3227742 ],
       [0.3221017 ],
       [0.32184812],
       [0.32183495],
       [0.32199755],
       [0.3220189 ],
       [0.32211962],
       [0.32236248],
       [0.3223473 ],
       [0.32225344],
       [0.3223403 ],
       [0.32266736],
       [0.32284823],
       [0.32273272],
       [0.3228705 ],
       [0.3228769 ],
       [0.322695  ],
       [0.32262418],
       [0.3221644 ],
       [0.32192165],
       [0.3216119 ],
       [0.32131314],
       [0.32150355],
       [0.32202712],
       [0.32198605],
       [0.32199943],
       [0.32175913],
       [0.321

In [20]:
predicted.reshape(382,13)

ValueError: cannot reshape array of size 382 into shape (382,13)